<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Fashion-MNIST with Keras and TensorBoard</H1></u></center>

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess) 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard

<h3>Labels</h3>

Each training and test example is assigned to one of the following labels:
<h4>
0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot 
</h4>

In [ ]:
train = pd.read_csv(r'../data/FMNIST/fashion-mnist_train.csv')
test = pd.read_csv(r'../data/FMNIST/fashion-mnist_test.csv')
train.head()

## Standardization of the data:

In [ ]:
train_data = np.array(train, dtype='float32')
test_data = np.array(test, dtype='float32')

x_train = train_data[:, 1:] / 255
y_train = train_data[:, 0]

x_test = test_data[:, 1:] / 255
y_test = test_data[:, 0]

## Splitting the data into train / val datasets

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2018)

## Visualizing an image:

In [ ]:
idx = [0,100,250,500,800,1500]
N = len(idx)
plt.figure(figsize=(15,15))
for i in range(N):
    plt.subplot(1,N, i+1)
    image = x_train[i, :].reshape((28, 28))
    plt.imshow(image)
    plt.title("Label: {}".format(y_train[idx[i]]))
    plt.axis('off')
plt.show()

## Reshaping the data:

In [ ]:
im_rows = 28
im_cols = 28
batch_size = 512
im_shape = (im_rows, im_cols, 1)

x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_val = x_val.reshape(x_val.shape[0], *im_shape)

print('x_train shape: {}'.format(x_train.shape))
print('x_test shape: {}'.format(x_test.shape))
print('x_validate shape: {}'.format(x_val.shape))

## Defining the model:

In [ ]:
cnn_model = Sequential([
    Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=im_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Dropout(0.3),   
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

In [ ]:
tensorboard = TensorBoard(
    log_dir=r'..\data\logs\{}'.format('cnn_1layer'),
    write_graph=True,
    write_grads=True,
    histogram_freq=1,
    write_images=True,
)

## Compiling the model:

In [ ]:
cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## Training the model:

In [ ]:
cnn_model.fit(
    x_train, y_train, batch_size=batch_size,
    epochs=10, verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[tensorboard]
)

## Reference:

https://github.com/zalandoresearch/fashion-mnist

https://www.kaggle.com/zalando-research/fashionmnist